In [78]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.metrics import (
    f1_score,
    roc_auc_score,
    roc_curve,
    average_precision_score
)
from sklearn.linear_model import LogisticRegression
import json

In [79]:
df = pd.read_csv("Models/Data/data.csv")
df

,timestamp,open,high,low,close,volume,vwap,transactions,otc
0,1712361600000,67856.51,69693.10,67379.00,68919.89,6124.067240,68283.2003,247970,NaN
1,1712448000000,68908.60,70312.00,68820.00,69350.34,7901.993047,69486.0920,325177,NaN
2,1712534400000,69350.36,72777.00,69036.00,71622.53,23223.579052,71576.5348,567352,NaN
3,1712620800000,71650.10,71761.37,68200.00,69117.01,21914.589601,69778.9649,539778,NaN
4,1712707200000,69115.79,71341.00,67463.07,70634.04,21928.775433,69027.2689,523393,NaN
...,...,...,...,...,...,...,...,...,...
358,1743292800000,82866.00,83697.00,81538.88,82379.98,4152.013942,82747.7186,291349,NaN
359,1743379200000,82325.40,83998.00,81265.16,82534.32,9345.129862,82632.3517,473988,NaN
360,1743465600000,82523.00,85566.53,82403.32,85170.37,11357.764516,84286.1218,496446,NaN
361,1743552000000,85166.92,89042.70,82141.50,82490.08,20950.876522,85421.1468,698121,NaN


In [80]:
def preprocess(trainDF, testDF):
    scaler = StandardScaler()
    trainDF = scaler.fit_transform(trainDF)
    testDF = scaler.fit_transform(testDF)
    return trainDF, testDF

In [84]:
def eval_gridsearch(clf, pgrid, xTrain, yTrain, xTest, yTest):

    cv = GridSearchCV(clf, param_grid=pgrid, cv=10)
    cv.fit(xTrain, yTrain)

    clf = cv.best_estimator_
    best_params = cv.best_params_

    yHat = cv.predict(xTest)
    yHat_proba = cv.predict_proba(xTest)[:,1]

    auc = roc_auc_score(yTest, yHat_proba)

    auprc = average_precision_score(yTest, yHat_proba)

    f1 = f1_score(yTest, yHat)

    fpr, tpr, _ = roc_curve(yTest, yHat_proba)

    return {'AUC': auc, 'AUPRC': auprc, 'F1': f1}, {'fpr': fpr, 'tpr': tpr}, best_params


In [85]:
def eval_randomsearch(clf, pgrid, xTrain, yTrain, xTest, yTest):
    permutations = np.prod([len(v) for v in pgrid.values()])

    cv = RandomizedSearchCV(clf, param_distributions=pgrid, n_iter=int(permutations*0.33), cv=10)
    cv.fit(xTrain, yTrain)

    clf = cv.best_estimator_
    best_params = cv.best_params_

    yHat = cv.predict(xTest)
    yHat_proba = cv.predict_proba(xTest)[:,1]

    auc = roc_auc_score(yTest, yHat_proba)

    auprc = average_precision_score(yTest, yHat_proba)

    f1 = f1_score(yTest, yHat)

    fpr, tpr, _ = roc_curve(yTest, yHat_proba)

    return {'AUC': auc, 'AUPRC': auprc, 'F1': f1}, {'fpr': fpr, 'tpr': tpr}, best_params


In [86]:
def get_parameter_grid(mName):
    if mName == 'DT':
        return {'max_depth': [2, 4], 'min_samples_leaf': [2, 3]}
    elif mName == 'LR (None)':
        return {'C': [0.001, 0.01, 0.1, 1, 10, 100, 1000], 'tol': [0.0001, 0.0004]}
    elif mName == 'LR (L1)':
        return {'C': [0.001, 0.01, 0.1, 1, 10, 100, 1000], 'tol': [0.0001, 0.0004]}
    elif mName == 'LR (L2)':
        return {'C': [0.001, 0.01, 0.1, 1, 10, 100, 1000], 'tol': [0.0001, 0.0004]}
    elif mName == 'KNN':
        return {'n_neighbors': [5, 10, 15], 'p': [1, 2]}
    elif mName == 'NN':
        return {'alpha':[0.0001, 0.0005, 0.001], 'hidden_layer_sizes': [(50, 50), (100, 100)]}


In [87]:
def eval_searchcv(clfName, clf, clfGrid,
                  xTrain, yTrain, xTest, yTest,
                  perfDict, rocDF, bestParamDict):
    # evaluate grid search and add to perfDict
    cls_perf, cls_roc, gs_p  = eval_gridsearch(clf, clfGrid, xTrain,
                                               yTrain, xTest, yTest)
    perfDict[clfName + " (Grid)"] = cls_perf
    # add to ROC DF
    rocRes = pd.DataFrame(cls_roc)
    rocRes["model"] = clfName
    rocDF = pd.concat([rocDF, rocRes], ignore_index=True)
    # evaluate random search and add to perfDict
    clfr_perf, _, rs_p  = eval_randomsearch(clf, clfGrid, xTrain,
                                            yTrain, xTest, yTest)
    perfDict[clfName + " (Random)"] = clfr_perf
    bestParamDict[clfName] = {"Grid": gs_p, "Random": rs_p}
    return perfDict, rocDF, bestParamDict

In [98]:
def main():
    # make classes
    X = df.drop(columns=['close', 'otc'])
    y = pd.DataFrame(columns=['price'])
    for i, row in df.iterrows():
        if row['open'] - row['close'] > 0:
            y.loc[i] = 1
        else:
            y.loc[i] = 0

    y.to_numpy().flatten()

    xTrain, xTest, yTrain, yTest = train_test_split(X, y, test_size=0.2)
    
    xTrain, xTest = preprocess(xTrain, xTest)

    perfDict = {}
    rocDF = pd.DataFrame()
    bestParamDict = {}

    print("Tuning Unregularized Logistic Regression --------")
    # logistic regression (unregularized)
    unregLrName = "LR (None)"
    unregLrGrid = get_parameter_grid(unregLrName)
    # fill in
    lrClf = LogisticRegression()
    perfDict, rocDF, bestParamDict = eval_searchcv(unregLrName, lrClf, unregLrGrid,
                                                   xTrain, yTrain, xTest, yTest,
                                                   perfDict, rocDF, bestParamDict)
    # logistic regression (L1)
    print("Tuning Logistic Regression (Lasso) --------")
    lassoLrName = "LR (L1)"
    lassoLrGrid = get_parameter_grid(lassoLrName)
    # fill in
    lassoClf = LogisticRegression(penalty='l1', solver='liblinear')
    perfDict, rocDF, bestParamDict = eval_searchcv(lassoLrName, lassoClf, lassoLrGrid,
                                                   xTrain, yTrain, xTest, yTest,
                                                   perfDict, rocDF, bestParamDict)
    # Logistic regression (L2)
    print("Tuning Logistic Regression (Ridge) --------")
    ridgeLrName = "LR (L2)"
    ridgeLrGrid = get_parameter_grid(ridgeLrName)
    # fill in
    ridgeClf = LogisticRegression(penalty='l2')
    perfDict, rocDF, bestParamDict = eval_searchcv(ridgeLrName, ridgeClf, ridgeLrGrid,
                                                   xTrain, yTrain, xTest, yTest,
                                                   perfDict, rocDF, bestParamDict)

    perfDF = pd.DataFrame.from_dict(perfDict, orient='index')
    print(perfDF)
    # save roc curves to data
    rocDF.to_csv('out'.rocOutput, index=False)
    # store the best parameters
    with open('best'.bestParamOutput, 'w') as f:
        json.dump(bestParamDict, f)

In [89]:
def holdout(model, xFeat, y, testSize):
    xTrain, xTest, yTrain, yTest = train_test_split(xFeat, y, test_size=testSize)
    resultDict = eval_randomsearch(model, xTrain, yTrain, xTest, yTest)
    return resultDict

In [100]:
main()

Tuning Unregularized Logistic Regression --------


c:\Users\thoma\anaconda3\envs\CS334\Lib\site-packages\sklearn\utils\validation.py:1408: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Users\thoma\anaconda3\envs\CS334\Lib\site-packages\sklearn\utils\validation.py:1408: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Users\thoma\anaconda3\envs\CS334\Lib\site-packages\sklearn\utils\validation.py:1408: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Users\thoma\anaconda3\envs\CS334\Lib\site-packages\sklearn\utils\validation.py:1408: DataConversionWarning: A column-vector y was passed when a 1d array was expected.

Tuning Logistic Regression (Lasso) --------


c:\Users\thoma\anaconda3\envs\CS334\Lib\site-packages\sklearn\utils\validation.py:1408: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Users\thoma\anaconda3\envs\CS334\Lib\site-packages\sklearn\svm\_base.py:1249: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
c:\Users\thoma\anaconda3\envs\CS334\Lib\site-packages\sklearn\utils\validation.py:1408: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Users\thoma\anaconda3\envs\CS334\Lib\site-packages\sklearn\utils\validation.py:1408: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(

Tuning Logistic Regression (Ridge) --------


c:\Users\thoma\anaconda3\envs\CS334\Lib\site-packages\sklearn\utils\validation.py:1408: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Users\thoma\anaconda3\envs\CS334\Lib\site-packages\sklearn\utils\validation.py:1408: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Users\thoma\anaconda3\envs\CS334\Lib\site-packages\sklearn\utils\validation.py:1408: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Users\thoma\anaconda3\envs\CS334\Lib\site-packages\sklearn\utils\validation.py:1408: DataConversionWarning: A column-vector y was passed when a 1d array was expected.

                         AUC     AUPRC        F1
LR (None) (Grid)    0.927177  0.926534  0.821918
LR (None) (Random)  0.924925  0.923427  0.821918
LR (L1) (Grid)      0.926426  0.924658  0.821918
LR (L1) (Random)    0.927177  0.926751  0.821918
LR (L2) (Grid)      0.927177  0.926534  0.821918
LR (L2) (Random)    0.923423  0.920696  0.821918


AttributeError: 'str' object has no attribute 'rocOutput'